In [275]:
import os
import numpy as np
import math
import sys
from time import time
import torchvision.transforms as transforms
from torchvision.utils import save_image
from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable
from torchvision.utils import save_image
import torch.nn as nn
import torch.nn.functional as F
import torch
import torchvision
import matplotlib.pyplot as plt

In [276]:
dataloader = torch.utils.data.DataLoader(
    torchvision.datasets.MNIST(
        "data/MNIST",
        train=True,
        download=True,
        transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=(0.5, ), std=(0.5,))])
    ),
    batch_size=128,
    shuffle=True,
)

In [277]:
class Opt(object):
    n_epochs = 40
    Diters = 5
    batchSize = 128
    lr = 0.00005
    n_cpu = 1
    latent_dim = 100
    imageSize = 32
    channels = 1
    n_critic = 5
    experiment = "experiments"
    sample_interval = 400

In [278]:
#Weight initialization
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

In [282]:
class Generator(nn.Module):
    def __init__(self, in_channel = 100,
             out_channel = 1,
             feature_map = 128,
             kernal_size = 3,
             image_size = 28):
        super(Generator, self).__init__()
        
        #Upsample random variable
        layers = nn.ModuleList()
        layers += [nn.ConvTranspose2d(in_channel, feature_map, 4, 1, 0)]
        layers += [nn.BatchNorm2d(feature_map)]
        layers += [nn.ReLU(True)]
        
        size = 4
        
        layers += [nn.ConvTranspose2d(feature_map, feature_map // 2, kernal_size, 2, 1, bias = False)]
        layers += [nn.BatchNorm2d(feature_map // 2)]
        layers += [nn.ReLU(True)]
        feature_map = feature_map // 2
        size = size * 2
        
        #Main G structure
        while size < image_size // 2:
            layers += [nn.ConvTranspose2d(feature_map, feature_map // 2, 4, 2, 1, bias = False)]
            layers += [nn.BatchNorm2d(feature_map // 2)]
            layers += [nn.ReLU(True)]
            feature_map = feature_map // 2
            size = size * 2
        
        #Final layer
        layers += [nn.ConvTranspose2d(feature_map, out_channel, 4, 2, 1, bias = False)]
        layers += [nn.Tanh()]
        
        self.g = nn.Sequential(*layers)
        
    def forward(self, z):
        return self.g(z)

In [283]:
class Discriminator(nn.Module):
    def __init__(self, in_channel = 1,
             out_channel = 1,
             feature_map = 32,
             kernal_size = 3,
             image_size = 28):
        super(Discriminator, self).__init__()
        
        #First layer
        layers = nn.ModuleList()
        layers += [nn.Conv2d(in_channel, feature_map, 4, 2, 1)]
        layers += [nn.BatchNorm2d(feature_map)]
        layers += [nn.LeakyReLU(0.2, inplace=True)]
        
        size = image_size / 2
        
        #Main D structure
        while size > 8:
            layers += [nn.Conv2d(feature_map, feature_map * 2, 4, 2, 1, bias = False)]
            layers += [nn.BatchNorm2d(feature_map * 2)]
            layers += [nn.LeakyReLU(0.2, inplace=True)]
            feature_map = feature_map * 2
            size = size / 2
            
        layers += [nn.Conv2d(feature_map, feature_map * 2, kernal_size, 2, 1, bias = False)]
        layers += [nn.BatchNorm2d(feature_map * 2)]
        layers += [nn.LeakyReLU(0.2, inplace=True)]
        feature_map = feature_map * 2
        size = size / 2
        
        #Final layer
        layers += [nn.Conv2d(feature_map, out_channel, 4, 1, 0, bias = False)]
        
        self.d = nn.Sequential(*layers)
        
    def forward(self, image):
        return self.d(image)  

In [284]:
device = torch.device("cuda" if (torch.cuda.is_available()) else "cpu")
generator = Generator().to(device)
discriminator = Discriminator().to(device)

In [285]:
opt = Opt()
cuda = True if torch.cuda.is_available() else False

generator_optimizer = torch.optim.RMSprop(generator.parameters(), lr=opt.lr)
discriminator_optimizer = torch.optim.RMSprop(discriminator.parameters(), lr=opt.lr)
Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

In [286]:
batches_done = 0
saved_imgs = []
input = torch.FloatTensor(opt.batchSize, 1, opt.imageSize, opt.imageSize)
noise = torch.FloatTensor(opt.batchSize, opt.latent_dim)
fixed_noise = torch.FloatTensor(opt.batchSize, opt.latent_dim).normal_(0, 1)
one = torch.FloatTensor([1])
mone = one * -1

gen_iterations = 0
for epoch in range(opt.n_epochs):
    for i, (imgs, _) in enumerate(dataloader):
        real_imgs = Variable(imgs.type(Tensor))
        real_imgs = real_imgs.to(device)
        for _ in range(opt.Diters):
            noise = torch.randn(opt.batchSize,opt.latent_dim, 1,1,device=device)
            fake = generator(noise).detach()
            errD_fake = torch.mean(discriminator(fake))
            errD_real = torch.mean(discriminator(real_imgs))
            errD = errD_fake - errD_real
            discriminator.zero_grad()
            errD.backward()
            discriminator_optimizer.step()
            for p in discriminator.parameters():
                p.data.clamp_(-0.01, 0.01)
                
        gen_fake = generator(noise)
        errG = -torch.mean(discriminator(gen_fake))        
        generator.zero_grad()
        errG.backward()
        generator_optimizer.step()
        gen_iterations += 1

        print('[%d/%d][%d/%d][%d] Loss_D: %f Loss_G: %f Loss_D_real: %f Loss_D_fake %f'
            % (epoch, opt.n_epochs, i, len(dataloader), gen_iterations,
            errD.item(), errG.item(), errD_real.item(), errD_fake.item()))
        if gen_iterations % 300 == 0:
            real_imgs = real_imgs.mul(0.5).add(0.5)
            save_image(real_imgs, '{0}/real_samples.png'.format(opt.experiment))
            with torch.no_grad():
                fake = generator(noise)
            fake.data = fake.data.mul(0.5).add(0.5)
            save_image(fake.data, '{0}/fake_samples_{1}.png'.format(opt.experiment, gen_iterations))

[0/40][5/469][1] Loss_D: -0.001752 Loss_G: 0.000785 Loss_D_real: 0.001042 Loss_D_fake -0.000711
[0/40][6/469][2] Loss_D: -0.001657 Loss_G: 0.000266 Loss_D_real: 0.001434 Loss_D_fake -0.000223
[0/40][7/469][3] Loss_D: -0.001777 Loss_G: 0.000147 Loss_D_real: 0.001673 Loss_D_fake -0.000104
[0/40][8/469][4] Loss_D: -0.001932 Loss_G: 0.000093 Loss_D_real: 0.001899 Loss_D_fake -0.000032
[0/40][9/469][5] Loss_D: -0.002041 Loss_G: -0.000010 Loss_D_real: 0.002119 Loss_D_fake 0.000078
[0/40][10/469][6] Loss_D: -0.002196 Loss_G: -0.000008 Loss_D_real: 0.002256 Loss_D_fake 0.000060
[0/40][11/469][7] Loss_D: -0.002428 Loss_G: 0.000106 Loss_D_real: 0.002382 Loss_D_fake -0.000046
[0/40][12/469][8] Loss_D: -0.002724 Loss_G: 0.000047 Loss_D_real: 0.002710 Loss_D_fake -0.000014
[0/40][13/469][9] Loss_D: -0.002884 Loss_G: 0.000083 Loss_D_real: 0.002850 Loss_D_fake -0.000034
[0/40][14/469][10] Loss_D: -0.003041 Loss_G: 0.000094 Loss_D_real: 0.003005 Loss_D_fake -0.000036
[0/40][15/469][11] Loss_D: -0.0035

[0/40][89/469][85] Loss_D: -0.113916 Loss_G: 0.058089 Loss_D_real: 0.060896 Loss_D_fake -0.053020
[0/40][90/469][86] Loss_D: -0.119285 Loss_G: 0.055532 Loss_D_real: 0.060271 Loss_D_fake -0.059015
[0/40][91/469][87] Loss_D: -0.118593 Loss_G: 0.060337 Loss_D_real: 0.062853 Loss_D_fake -0.055740
[0/40][92/469][88] Loss_D: -0.122157 Loss_G: 0.057823 Loss_D_real: 0.060627 Loss_D_fake -0.061530
[0/40][93/469][89] Loss_D: -0.124164 Loss_G: 0.062717 Loss_D_real: 0.065146 Loss_D_fake -0.059018
[0/40][94/469][90] Loss_D: -0.128004 Loss_G: 0.059880 Loss_D_real: 0.063216 Loss_D_fake -0.064788
[0/40][95/469][91] Loss_D: -0.125843 Loss_G: 0.065970 Loss_D_real: 0.065343 Loss_D_fake -0.060500
[0/40][96/469][92] Loss_D: -0.131885 Loss_G: 0.067568 Loss_D_real: 0.067540 Loss_D_fake -0.064346
[0/40][97/469][93] Loss_D: -0.134074 Loss_G: 0.067315 Loss_D_real: 0.070312 Loss_D_fake -0.063762
[0/40][98/469][94] Loss_D: -0.131813 Loss_G: 0.062078 Loss_D_real: 0.063995 Loss_D_fake -0.067818
[0/40][99/469][95] L

[0/40][172/469][168] Loss_D: -0.178942 Loss_G: 0.079977 Loss_D_real: 0.074311 Loss_D_fake -0.104631
[0/40][173/469][169] Loss_D: -0.183190 Loss_G: 0.102747 Loss_D_real: 0.096912 Loss_D_fake -0.086278
[0/40][174/469][170] Loss_D: -0.191651 Loss_G: 0.096320 Loss_D_real: 0.093733 Loss_D_fake -0.097918
[0/40][175/469][171] Loss_D: -0.190080 Loss_G: 0.101560 Loss_D_real: 0.097027 Loss_D_fake -0.093053
[0/40][176/469][172] Loss_D: -0.189859 Loss_G: 0.091566 Loss_D_real: 0.087281 Loss_D_fake -0.102578
[0/40][177/469][173] Loss_D: -0.182240 Loss_G: 0.103625 Loss_D_real: 0.100477 Loss_D_fake -0.081763
[0/40][178/469][174] Loss_D: -0.184952 Loss_G: 0.086276 Loss_D_real: 0.083149 Loss_D_fake -0.101803
[0/40][179/469][175] Loss_D: -0.188245 Loss_G: 0.105744 Loss_D_real: 0.101634 Loss_D_fake -0.086611
[0/40][180/469][176] Loss_D: -0.189702 Loss_G: 0.088605 Loss_D_real: 0.084503 Loss_D_fake -0.105199
[0/40][181/469][177] Loss_D: -0.193141 Loss_G: 0.102257 Loss_D_real: 0.099763 Loss_D_fake -0.093378


[0/40][254/469][250] Loss_D: -0.189348 Loss_G: 0.102760 Loss_D_real: 0.100263 Loss_D_fake -0.089085
[0/40][255/469][251] Loss_D: -0.188919 Loss_G: 0.090916 Loss_D_real: 0.090843 Loss_D_fake -0.098076
[0/40][256/469][252] Loss_D: -0.190614 Loss_G: 0.105279 Loss_D_real: 0.102596 Loss_D_fake -0.088017
[0/40][257/469][253] Loss_D: -0.191838 Loss_G: 0.088626 Loss_D_real: 0.091780 Loss_D_fake -0.100058
[0/40][258/469][254] Loss_D: -0.187838 Loss_G: 0.103490 Loss_D_real: 0.102783 Loss_D_fake -0.085055
[0/40][259/469][255] Loss_D: -0.190824 Loss_G: 0.086889 Loss_D_real: 0.091175 Loss_D_fake -0.099649
[0/40][260/469][256] Loss_D: -0.186178 Loss_G: 0.105228 Loss_D_real: 0.103161 Loss_D_fake -0.083017
[0/40][261/469][257] Loss_D: -0.186619 Loss_G: 0.078276 Loss_D_real: 0.083123 Loss_D_fake -0.103496
[0/40][262/469][258] Loss_D: -0.183059 Loss_G: 0.104126 Loss_D_real: 0.104654 Loss_D_fake -0.078405
[0/40][263/469][259] Loss_D: -0.189971 Loss_G: 0.081001 Loss_D_real: 0.084581 Loss_D_fake -0.105390


[0/40][336/469][332] Loss_D: -0.154709 Loss_G: 0.094745 Loss_D_real: 0.090328 Loss_D_fake -0.064381
[0/40][337/469][333] Loss_D: -0.165955 Loss_G: 0.073011 Loss_D_real: 0.067292 Loss_D_fake -0.098663
[0/40][338/469][334] Loss_D: -0.170846 Loss_G: 0.098159 Loss_D_real: 0.092961 Loss_D_fake -0.077885
[0/40][339/469][335] Loss_D: -0.168306 Loss_G: 0.076958 Loss_D_real: 0.072930 Loss_D_fake -0.095376
[0/40][340/469][336] Loss_D: -0.167440 Loss_G: 0.099710 Loss_D_real: 0.092486 Loss_D_fake -0.074954
[0/40][341/469][337] Loss_D: -0.169887 Loss_G: 0.078008 Loss_D_real: 0.072277 Loss_D_fake -0.097609
[0/40][342/469][338] Loss_D: -0.160708 Loss_G: 0.094473 Loss_D_real: 0.090410 Loss_D_fake -0.070299
[0/40][343/469][339] Loss_D: -0.164667 Loss_G: 0.072373 Loss_D_real: 0.067775 Loss_D_fake -0.096892
[0/40][344/469][340] Loss_D: -0.160794 Loss_G: 0.097832 Loss_D_real: 0.091958 Loss_D_fake -0.068836
[0/40][345/469][341] Loss_D: -0.169656 Loss_G: 0.079996 Loss_D_real: 0.073574 Loss_D_fake -0.096082


...

[39/40][85/469][18372] Loss_D: -0.015391 Loss_G: -0.040498 Loss_D_real: -0.039207 Loss_D_fake -0.054597
[39/40][86/469][18373] Loss_D: -0.015904 Loss_G: 0.028052 Loss_D_real: 0.003720 Loss_D_fake -0.012184
[39/40][87/469][18374] Loss_D: -0.013130 Loss_G: -0.035290 Loss_D_real: -0.036562 Loss_D_fake -0.049692
[39/40][88/469][18375] Loss_D: -0.018218 Loss_G: 0.036586 Loss_D_real: 0.043629 Loss_D_fake 0.025411
[39/40][89/469][18376] Loss_D: -0.017622 Loss_G: -0.013584 Loss_D_real: -0.017918 Loss_D_fake -0.035540
[39/40][90/469][18377] Loss_D: -0.013976 Loss_G: 0.039501 Loss_D_real: 0.068196 Loss_D_fake 0.054220
[39/40][91/469][18378] Loss_D: -0.016097 Loss_G: -0.028736 Loss_D_real: -0.013680 Loss_D_fake -0.029776
[39/40][92/469][18379] Loss_D: -0.015339 Loss_G: 0.031958 Loss_D_real: 0.066173 Loss_D_fake 0.050834
[39/40][93/469][18380] Loss_D: -0.014158 Loss_G: -0.071859 Loss_D_real: -0.027370 Loss_D_fake -0.041527
[39/40][94/469][18381] Loss_D: -0.017932 Loss_G: -0.038968 Loss_D_real: -0.

[39/40][165/469][18452] Loss_D: -0.017090 Loss_G: -0.068590 Loss_D_real: -0.046212 Loss_D_fake -0.063302
[39/40][166/469][18453] Loss_D: -0.014503 Loss_G: -0.030252 Loss_D_real: -0.049523 Loss_D_fake -0.064026
[39/40][167/469][18454] Loss_D: -0.016554 Loss_G: 0.060412 Loss_D_real: 0.043944 Loss_D_fake 0.027390
[39/40][168/469][18455] Loss_D: -0.017808 Loss_G: -0.024862 Loss_D_real: -0.044625 Loss_D_fake -0.062433
[39/40][169/469][18456] Loss_D: -0.012952 Loss_G: 0.035533 Loss_D_real: 0.068949 Loss_D_fake 0.055997
[39/40][170/469][18457] Loss_D: -0.021618 Loss_G: -0.028204 Loss_D_real: -0.002961 Loss_D_fake -0.024579
[39/40][171/469][18458] Loss_D: -0.023174 Loss_G: 0.062687 Loss_D_real: 0.058255 Loss_D_fake 0.035081
[39/40][172/469][18459] Loss_D: -0.016779 Loss_G: 0.024566 Loss_D_real: -0.001694 Loss_D_fake -0.018473
[39/40][173/469][18460] Loss_D: -0.017825 Loss_G: -0.040644 Loss_D_real: -0.039289 Loss_D_fake -0.057114
[39/40][174/469][18461] Loss_D: -0.016604 Loss_G: -0.048299 Loss_

[39/40][245/469][18532] Loss_D: -0.020347 Loss_G: 0.066752 Loss_D_real: 0.031262 Loss_D_fake 0.010915
[39/40][246/469][18533] Loss_D: -0.014981 Loss_G: -0.022672 Loss_D_real: -0.040253 Loss_D_fake -0.055234
[39/40][247/469][18534] Loss_D: -0.016543 Loss_G: 0.039868 Loss_D_real: 0.029120 Loss_D_fake 0.012577
[39/40][248/469][18535] Loss_D: -0.015267 Loss_G: -0.031708 Loss_D_real: -0.012297 Loss_D_fake -0.027564
[39/40][249/469][18536] Loss_D: -0.015613 Loss_G: 0.063902 Loss_D_real: 0.051342 Loss_D_fake 0.035729
[39/40][250/469][18537] Loss_D: -0.018449 Loss_G: -0.049377 Loss_D_real: -0.035226 Loss_D_fake -0.053675
[39/40][251/469][18538] Loss_D: -0.015025 Loss_G: 0.054105 Loss_D_real: 0.029527 Loss_D_fake 0.014502
[39/40][252/469][18539] Loss_D: -0.016257 Loss_G: -0.018290 Loss_D_real: -0.015657 Loss_D_fake -0.031914
[39/40][253/469][18540] Loss_D: -0.013957 Loss_G: 0.033682 Loss_D_real: 0.068723 Loss_D_fake 0.054766
[39/40][254/469][18541] Loss_D: -0.015457 Loss_G: -0.035247 Loss_D_rea

[39/40][325/469][18612] Loss_D: -0.012992 Loss_G: 0.053030 Loss_D_real: 0.075102 Loss_D_fake 0.062110
[39/40][326/469][18613] Loss_D: -0.019168 Loss_G: 0.007085 Loss_D_real: -0.010707 Loss_D_fake -0.029875
[39/40][327/469][18614] Loss_D: -0.015474 Loss_G: 0.052301 Loss_D_real: 0.048853 Loss_D_fake 0.033379
[39/40][328/469][18615] Loss_D: -0.018575 Loss_G: -0.016622 Loss_D_real: -0.009523 Loss_D_fake -0.028099
[39/40][329/469][18616] Loss_D: -0.013747 Loss_G: 0.061338 Loss_D_real: 0.062946 Loss_D_fake 0.049198
[39/40][330/469][18617] Loss_D: -0.020629 Loss_G: -0.042977 Loss_D_real: 0.002816 Loss_D_fake -0.017813
[39/40][331/469][18618] Loss_D: -0.015829 Loss_G: 0.057027 Loss_D_real: 0.055387 Loss_D_fake 0.039557
[39/40][332/469][18619] Loss_D: -0.016500 Loss_G: -0.025281 Loss_D_real: -0.008655 Loss_D_fake -0.025155
[39/40][333/469][18620] Loss_D: -0.016107 Loss_G: 0.057410 Loss_D_real: 0.018587 Loss_D_fake 0.002479
[39/40][334/469][18621] Loss_D: -0.017988 Loss_G: -0.025561 Loss_D_real:

[39/40][405/469][18692] Loss_D: -0.019697 Loss_G: 0.062441 Loss_D_real: 0.018542 Loss_D_fake -0.001155
[39/40][406/469][18693] Loss_D: -0.018933 Loss_G: 0.014452 Loss_D_real: 0.025658 Loss_D_fake 0.006725
[39/40][407/469][18694] Loss_D: -0.011581 Loss_G: -0.036092 Loss_D_real: -0.043440 Loss_D_fake -0.055021
[39/40][408/469][18695] Loss_D: -0.014927 Loss_G: 0.049854 Loss_D_real: 0.065300 Loss_D_fake 0.050373
[39/40][409/469][18696] Loss_D: -0.017165 Loss_G: -0.062875 Loss_D_real: -0.034069 Loss_D_fake -0.051234
[39/40][410/469][18697] Loss_D: -0.010532 Loss_G: -0.023238 Loss_D_real: -0.044675 Loss_D_fake -0.055207
[39/40][411/469][18698] Loss_D: -0.012913 Loss_G: 0.052796 Loss_D_real: 0.063866 Loss_D_fake 0.050954
[39/40][412/469][18699] Loss_D: -0.019154 Loss_G: -0.011674 Loss_D_real: -0.010173 Loss_D_fake -0.029327
[39/40][413/469][18700] Loss_D: -0.016806 Loss_G: 0.020422 Loss_D_real: 0.027432 Loss_D_fake 0.010626
[39/40][414/469][18701] Loss_D: -0.017420 Loss_G: -0.069307 Loss_D_re